In [1]:
from itertools import product
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
# from torch.nn.functional import relu
# from torch.nn.functional import tanh
# from torch.nn.functional import sigmoid
from scipy.fftpack import dct
from scipy.signal import convolve2d
from skimage.measure import block_reduce
from sklearn import svm

## Reduce the data size by shrinking the image and allow symbolic dynamics to occur (based on doppler-effect).

### Load the data

In [2]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
)
data_len = {'train': len(train_data), 'test': len(test_data)}

loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
}

### Symbolic dynamics through iteration and non-linearity.
Through multiple iterations apply the doppler effect to the DCT of the images and apply convolution.  
Save the results in a dataframe for easy loading.

In [3]:
def relu(x):
    return np.maximum(0, x)
relu = np.vectorize(relu)
# sigmoid = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
tanh = np.vectorize(np.tanh)

In [43]:
def apply_dynamics(sample, iters, f_act, vel, v_o, v_m, kernels, conv=False, pool=False):
    final_sample = np.zeros(0)

    for k in kernels:
        working_sample = np.copy(sample)

        for i in range(iters):
            v_s = vel(i, iters)
            # apply doppler effect to sample
            working_sample = ((v_m + v_o) / (v_m + v_s)) * working_sample  # doppler-effect
            working_sample = f_act(working_sample)  # activation function

            if conv:
                working_sample = convolve2d(working_sample, k, mode='valid')  # convolution

            if pool and i % 4 == 1:
                working_sample = block_reduce(working_sample, (2, 2), np.mean, cval=0.5)  # pooling

        final_sample = np.concat((final_sample, working_sample.reshape(working_sample.size) * 100))  # combine results of kernels

    return final_sample

###### Visualization
Changing `iters` determines the size of the output (if conv and/or pooling are on).

In [34]:
samples, labels = next(iter(loaders['train']))
sample = samples[0][0]

# vel_s is negative if moving towards observer
def v1(x, n):
    return -(x + 1) / (n / 3)
def v2(x, n):
    return x
def v3(x, n):
    return 1.5

v_o = 0  # positive if moving towards source
v_m = 5.022  # small tail to avoid division errors.
kernels = np.array([
    [[0.25, 0.25],
     [0.25, 0.25]],  # Avg kernel
    [[0.5, 0.5],
     [0, 0]],  # Horizontal kernel
    [[0.5, 0],
     [0.5, 0]],  # Vertical kernel
    [[0.5, 0],
     [0, 0.5]],  # Diagonal down kernel
    [[0, 0.5],
     [0.5, 0]]  # Diagonal up kernel
])

In [45]:
iters = 7
f_act = relu  # relu, tanh
conv = True
pool = True

freq_sample = dct(dct(sample.numpy().T, norm='ortho').T, norm='ortho')  # decompose sample
print(f"freq_sample {freq_sample.shape}\n")  # {np.round(freq_sample, 2)}")
result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernels, conv=conv, pool=pool)
print(f"result {result.shape}\n{np.round(result, 2)}")

freq_sample (28, 28)

result (80,)
[ 0.52  0.51  0.35  5.85  0.19  0.23  0.21  5.81  0.09  0.09  0.09  5.76
  5.75  5.75  5.75 10.01  0.1   0.09  0.08  5.75  0.06  0.07  0.07  5.75
  0.08  0.07  0.06  5.75 11.42 11.42 11.43 14.27  0.4   0.18  0.08 11.43
  0.25  0.15  0.09 11.44  0.08  0.08  0.1  11.46  5.74  5.75  5.78 14.29
  0.48  0.79  0.44  5.83  0.13  0.16  0.29  5.89  0.09  0.09  0.08  5.77
  5.75  5.75  5.75  8.59  0.48  0.42  0.29  5.83  0.22  0.22  0.16  5.77
  0.09  0.09  0.09  5.74  5.75  5.76  5.74 11.43]


###### Convert all data
Changing `iters` determines the size of the output.

In [46]:
# iters = 18
# f_act = relu
# conv = False
# pool = False

df_results = pd.DataFrame(columns=['data', 'label', 'train'])
for dset in ["train", "test"]:
    batch_size = 100
    batches = data_len[dset] / batch_size
    print(f"Begin reducing {dset}")
    for b, (images, labels) in enumerate(loaders[dset]):
        if b == batches:
            break
        for i in range(batch_size):
            freq_sample = dct(dct(images[i].numpy()[0].T, norm='ortho').T, norm='ortho')  # decompose sample
            result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernels, conv=conv, pool=pool)
            result_str = np.array2string(result, separator=',')
            df_results = pd.concat([df_results, pd.DataFrame([[result_str, labels[i].numpy(), dset == "train"]], columns=df_results.columns)], ignore_index=True)

        if (b + 1) % 30 == 0:
            print('Reduced [{}/{}] {} batches for {} iterations'
                  .format(b + 1, batches, dset, iters))

print(df_results.shape)
df_results.to_csv(f'freq_RedData/multi5_freq_RedData_{iters}i_conv{"_pool" if pool else ""}_{f_act.__name__}.csv', index=False)

Begin reducing train
Reduced [30/600.0] train batches for 7 iterations
Reduced [60/600.0] train batches for 7 iterations
Reduced [90/600.0] train batches for 7 iterations
Reduced [120/600.0] train batches for 7 iterations
Reduced [150/600.0] train batches for 7 iterations
Reduced [180/600.0] train batches for 7 iterations
Reduced [210/600.0] train batches for 7 iterations
Reduced [240/600.0] train batches for 7 iterations
Reduced [270/600.0] train batches for 7 iterations
Reduced [300/600.0] train batches for 7 iterations
Reduced [330/600.0] train batches for 7 iterations
Reduced [360/600.0] train batches for 7 iterations
Reduced [390/600.0] train batches for 7 iterations
Reduced [420/600.0] train batches for 7 iterations
Reduced [450/600.0] train batches for 7 iterations
Reduced [480/600.0] train batches for 7 iterations
Reduced [510/600.0] train batches for 7 iterations
Reduced [540/600.0] train batches for 7 iterations
Reduced [570/600.0] train batches for 7 iterations
Reduced [600/